In [70]:
import ast
import os
import sys
import json
import pickle
import nmslib
import numpy as np
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from gensim.models.fasttext import FastText

sys.path.append('../')
from utils.dataset_prepare import clean_text, lemmatization, correct_text, join_columns, get_tokens

### Подготовка датасета

In [71]:
dataset_path = 'all_recepies_inter.csv'

dataset_dir = '../datasets'
prepared_dataset_path = os.path.join(dataset_dir, 'recipes.json')

In [72]:
os.makedirs(dataset_dir, exist_ok=True)

In [73]:
df = pd.read_csv(dataset_path, sep='\t', usecols=('name', 'composition', 'Инструкции')).rename(columns={"Инструкции": "instructions"})

In [74]:
def convert_composition(composition: str):
    composition = ast.literal_eval(composition)
    foods = []
    for d in composition:
        foods.append(list(d.keys())[0])
    return ' '.join(foods)

In [75]:
df['composition'] = df['composition'].apply(convert_composition)

In [76]:
df.tail(3)

,name,composition,instructions
27881,Самый зеленый салат,Мангольд Яблоки «гренни-смит» Огурцы Оливки Зе...,"1. Чеснок натереть, сыр фета раскрошить.\r\n2...."
27882,Теплый салат с тыквой и брынзой под…,Тыква Сыр брынза Кедровые орехи Кунжутные семе...,"1. Тыкву нарезаем кубиками, добавляем оливково..."
27883,Салат из раковых шеек в авокадо,Авокадо Груши Сыр Зеленый лук Раковые шейки Тв...,"1. Авокадо разрезать на две половинки, удалить..."


Удалим рецепты в которых есть ссылки на внешние ресурсы.

In [77]:
df = df[~df['instructions'].str.contains('http', na=False)]

In [78]:
unique_val = df['name'].nunique()
unique_val

27776

In [79]:
df['name'].value_counts()

name
Оссобуко                                       3
Новогодний салат "Елка"                        3
Клубничный сорбет                              3
Шоколадный мусс                                3
Тыквенный пирог                                3
                                              ..
Картофельная запеканка с капустой и грибами    1
Грудинка в пакете                              1
Курица в вине с грибами                        1
Курица с лимоном и медом от Гордона Рамзи      1
Утиная печень с грибами                        1
Name: count, Length: 27776, dtype: int64

Удалим дубликаты инструкций.

In [80]:
df = df.drop_duplicates(subset=['name'])
df = df.drop_duplicates(subset=['instructions'])

In [81]:
df.isnull().sum()

name            0
composition     0
instructions    0
dtype: int64

#### Объединим столбцы 'name' и 'composition'

In [82]:
df = df.reset_index(drop=True)

In [83]:
join_columns(df, 'dish_name', ['name','composition'])

100%|██████████| 27753/27753 [00:00<00:00, 135352.05it/s]


In [84]:
df.head()

,name,composition,instructions,dish_name
0,рассольник классический с перловкой и солеными...,Перловка Соленые огурцы Морковь Лук Чеснок Сте...,Подготовить указанные ингредиенты для приготов...,рассольник классический с перловкой и солеными...
1,Суп пюре из белокочаной капусты,Капуста белокочанная Картошка Лук репчатый Мор...,"Необходимые ингредиенты\r\nНарезаем лук, морко...",суп пюре из белокочаной капусты капуста белоко...
2,Постные щи из квашеной капусты,Капуста квашеная Мак пищевой Морковь Лук репча...,"Честно признаюсь, у меня не было репы на момен...",постные щи из квашеной капусты капуста квашена...
3,Тюря- простой суп быстро и вкусно,Квас Лук репчатый Черный хлеб Чеснок Зелёный л...,"\r\nНачинаем мы приготовление тюри с того, что...",тюря- простой суп быстро и вкусно квас лук реп...
4,Фасолевый суп из красной фасоли,Вода Картошка Морковь Фасоль Томатная паста Пе...,Подготовить ингредиенты. Для приготовления суп...,фасолевый суп из красной фасоли вода картошка ...


Выполним предобработку текста перед тем как обучить FastText 
1. Уберем из текста стоп-слова, 
2. выполним лематизацию текста.

In [85]:
df['dish_name'] = df['dish_name'].apply(lambda x: clean_text(x))

In [86]:
%%time 
df['dish_name'] = df['dish_name'].apply(lambda x: lemmatization(x))

CPU times: user 1.34 s, sys: 280 µs, total: 1.35 s
Wall time: 1.34 s


In [87]:
df['dish_name'] = df['dish_name'].apply(lambda x: list(set(x.split(','))))

In [88]:
df.head()

,name,composition,instructions,dish_name
0,рассольник классический с перловкой и солеными...,Перловка Соленые огурцы Морковь Лук Чеснок Сте...,Подготовить указанные ингредиенты для приготов...,"[рассольник, паста, томатная, картошка, стебел..."
1,Суп пюре из белокочаной капусты,Капуста белокочанная Картошка Лук репчатый Мор...,"Необходимые ингредиенты\r\nНарезаем лук, морко...","[масло, вода, лист, картошка, лавровый, оливко..."
2,Постные щи из квашеной капусты,Капуста квашеная Мак пищевой Морковь Лук репча...,"Честно признаюсь, у меня не было репы на момен...","[масло, пищевой, щи, репа, квашеной, капусты, ..."
3,Тюря- простой суп быстро и вкусно,Квас Лук репчатый Черный хлеб Чеснок Зелёный л...,"\r\nНачинаем мы приготовление тюри с того, что...","[хлеб, зелень, кинзы, простой, репчатый, квас,..."
4,Фасолевый суп из красной фасоли,Вода Картошка Морковь Фасоль Томатная паста Пе...,Подготовить ингредиенты. Для приготовления суп...,"[петрушка, фасоли, паста, молотый, томатная, м..."


In [90]:
df.to_json(prepared_dataset_path, index=False, orient='table', force_ascii=False)

### Обучение модели векторизатора (FastText).

In [91]:
vectorizer_model_path = '../models/fasttext.model'
vector_db = '../datasets/weighted_doc_vects.p'

In [92]:
tok_text = list(df['dish_name'].values)

In [93]:
ft_model = FastText(
    sg=1, # use skip-gram: usually gives better results
    vector_size=200, # embedding dimension (default)
    window=10, # window size: 10 tokens before and 10 tokens after to get wider context
    min_count=5, # only consider tokens with at least n occurrences in the corpus
    negative=15, # negative subsampling: bigger than default to sample negative examples more
    min_n=2, # min character n-gram
    max_n=6 # max character n-gram
)
ft_model.build_vocab(tok_text) # tok_text is our tokenized input text - a list of lists relating to docs and tokens respectivley

In [94]:
ft_model.train(
    tok_text,
    epochs=6,
    total_examples=ft_model.corpus_count, 
    total_words=ft_model.corpus_total_words)

ft_model.save(vectorizer_model_path)

In [95]:
# Здесь рекомендации посмотрены по иному принципу. Сперва выполняется поиск конкрентого названия фильма, 
# а затем на основе найденного фильма система рекомендует выдает похожие на него.

### Создание базы векторов для всех рецептов.

In [96]:
ft_model = FastText.load(vectorizer_model_path)

In [97]:
bm25 = BM25Okapi(tok_text)

In [98]:
weighted_doc_vects = []

for i, doc in tqdm(enumerate(tok_text), total=len(tok_text)):
  doc_vector = []
  for word in doc:
    vector = ft_model.wv[word]
    #note for newer versions of fasttext you may need to replace ft_model[word] with ft_model.wv[word]
    weight = ((bm25.idf[word] * ((bm25.k1 + 1.0)*bm25.doc_freqs[i][word])) /
    (bm25.k1 * (1.0 - bm25.b + bm25.b *(bm25.doc_len[i]/bm25.avgdl))+bm25.doc_freqs[i][word]))
    weighted_vector = vector * weight
    doc_vector.append(weighted_vector) # Создается вектор для каждого долкумента.
  doc_vector_mean = np.mean(doc_vector,axis=0)
  weighted_doc_vects.append(doc_vector_mean)

100%|██████████| 27753/27753 [00:02<00:00, 12797.39it/s]


In [99]:
with open(vector_db, "wb") as file:
    pickle.dump(weighted_doc_vects, file)

In [100]:
with open(vector_db, "rb" ) as f:
  weighted_doc_vects = pickle.load(f)
# create a random matrix to index
data = np.vstack(weighted_doc_vects)

# initialize a new index, using a HNSW index on Cosine Similarity - can take a couple of mins
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(data)
index.createIndex({'post': 2}, print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
****************************************************

In [101]:
def find_recipes(text: str, count: int = 5, max_distance: int = 0.2):
  text_tokens = get_tokens(text)
  query = [ft_model.wv[vec] for vec in text_tokens]
  query = np.mean(query, axis=0)

  ids, distances = index.knnQuery(query, k=count)
  for i, distance in zip(ids, distances):
    if distance <= max_distance:
      print(f"distance: {distance:.2f},\t {df['name'].values[i]}")
    # На основе полученных заголовков рецептов. выполняем поиск самого ррецепта.

In [102]:
query = 'Расскажи, как приготовить яблочный пирог?'

find_recipes(query, 10)

distance: 0.09,	 Пирог яблочный
distance: 0.10,	 Простой яблочный пирог
distance: 0.10,	 Яблочный пирог в мультиварке
distance: 0.10,	 Традиционный американский яблочный пирог
distance: 0.10,	 Классический рецепт перевернутого пирога татен
distance: 0.10,	 Яблочный пирог
distance: 0.10,	 Эльзасский яблочный пирог
distance: 0.11,	 Скандинавский яблочный пирог
distance: 0.11,	 Яблочный пирог со штрейзелем
distance: 0.11,	 Турноверы с яблоками


In [103]:
query = 'Пирожки с грибами'

find_recipes(query, 50, max_distance = 0.15)

distance: 0.13,	 Картофельные пирожки с грибами
distance: 0.14,	 Драники с грибами


Следующие ответы странные. Необходимо отрабатывать случаи когда когда запрос не касается рецептов. 

In [104]:
query = 'Здравствуй, Кибер-бабушка!'

find_recipes(query, 10)

distance: 0.07,	 Банья-кауда с овощами
distance: 0.07,	 Бургер «Балибей»
distance: 0.07,	 Суп глубокого юга Гамбо с бамией
distance: 0.08,	 Салат из авокадо, яиц, бекона и печеных…
distance: 0.08,	 Салат из эммера с овощами
distance: 0.08,	 Куббургер
distance: 0.08,	 Кенигсбергские клопсы
distance: 0.08,	 Чикенбургер с голландским соусом
distance: 0.08,	 Балык экмек
distance: 0.08,	 Мясной каламбур из курицы и говядины


In [105]:
query = 'Спасибо'

find_recipes(query, 10, max_distance=0.15)

distance: 0.10,	 Морские гребешки в соусе из грибов и спаржи
distance: 0.11,	 Зеленый суп из молодого шпината и спаржи с…
distance: 0.11,	 Карпаччо из осьминога с поке из авокадо и…
distance: 0.11,	 Ризотто из риса венере с авокадо и карпаччо…
distance: 0.11,	 Густой томатный суп с лососем, тигровыми…
distance: 0.12,	 Тобан из морепродуктов
distance: 0.12,	 Карпаччо из лосося с азиатским соусом и…
distance: 0.12,	 Палтус в аква пацца из ресторана Christian
distance: 0.12,	 Киноа с лососем, спаржей и вешенками
distance: 0.12,	 Теплый салат из морепродуктов с зелеными…
